In [17]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import random
from sklearn.preprocessing import OneHotEncoder 
from scipy.sparse import coo_matrix, hstack,vstack
import time
from sklearn.utils import shuffle

In [18]:
df=pd.read_csv("NetflixProcessedDataFile.csv").drop_duplicates()
print(df.iloc[::5000000, :])#START:STOP:STEP (5000000)

           Unnamed: 0  User_Id  Rating  Movie_Id
0                   1  1488844     3.0         1
5000000       5000996   501954     2.0       996
10000000     10001962   404654     5.0      1962
15000000     15002876   886608     2.0      2876
20000000     20003825  1193835     2.0      3825
25000000     25004661  1899206     3.0      4661
30000000     30005496   154804     4.0      5496
35000000     35006274  2078749     5.0      6274
40000000     40007057   450763     5.0      7057
45000000     45007991   102092     3.0      7991
50000000     50009023   220298     5.0      9023
55000000     55010042   550530     5.0     10042
60000000     60011038   222570     3.0     11038
65000000     65011875  1273080     5.0     11875
70000000     70012676  2026970     5.0     12676
75000000     75013582   506044     4.0     13582
80000000     80014453   353605     2.0     14453
85000000     85015116   664606     3.0     15116
90000000     90016008  2213715     3.0     16008
95000000     9501687

In [19]:
# Get one hot encoding of column User_id
#one_hot_users = pd.get_dummies(df1['User_Id'])
#one_hot_users
# Drop column User_Id as it is now encoded
#df1 = df1.drop('User_Id',axis = 1)
# Join the encoded df
#df1 = df1.join(one_hot_users)
#df1  

#This approach with dummies wouldnt work, because dataset is too large

In [20]:
# importing one hot encoder from sklearn 
encoder = OneHotEncoder(categories='auto') 
#X-train matrix for encoder to learn categories
#enc.fit(X)  #process of encoder learning categories
one_hot_user_matrix = encoder.fit_transform(np.asarray(df['User_Id']).reshape(-1,1)) 
one_hot_movie_matrix = encoder.fit_transform(np.asarray(df['Movie_Id']).reshape(-1,1)) 
print(one_hot_movie_matrix.shape) #amount of rows=amount of ratings , amount of columns=amount of unique user_ids

(100480507, 17770)


In [21]:
X=hstack([one_hot_user_matrix,one_hot_movie_matrix]) #24053764x475257 matrix joined user_id and movie_id 2 groups
#475257=amount of users+amount of films
#24053764=amount of non null ratings
#create ratings vector 
ratings=np.asarray(df['Rating']).reshape(-1,1)
#ratings=df['Rating']
print(X.shape)
print(ratings.shape)

(100480507, 497959)
(100480507, 1)


In [22]:
# def svd_predict(X_input,omega,omega0,V,sum_const):
#     #linear_part=omega0+X.dot(omega)
#     #V_part=(X.dot(V)**2)-X.power(2).dot(V**2) # (24053764,3)
#     #return omega0+X.dot(omega)+(0.5*np.sum(np.square(sum_const)-X.power(2).dot(np.square(V)),axis=1))# (24053764,1)
#     return omega0 + X_input.dot(omega) + (0.5 * np.sum((sum_const**2) - (X_input.power(2)).dot(V**2), axis=1)) 
def svd_predict(X,omega,omega0,V,sum_const):
    #omega (475257,1)
    #omega0 goes to (24053764,1) in sum 
    # V (475257,3)
    # X (24053764, 475257)
    prod=X.dot(omega)#(24053764,1)
    linear_part=omega0+prod
    #V_part=np.square(sum_const)-X.power(2).dot(np.square(V)) # (24053764,3)
    V_part=(sum_const**2)-X.power(2).dot((V)**2) 
    sqr_part=(0.5*np.sum(V_part,axis=1)).reshape(-1,1) # (24053764,1)
    return linear_part+sqr_part # (24053764,1)
    
def root_mean_square_error(ratings_predict,ratings_actual):
    #ratings_predict=svd_predict(X,omega,omega0,V)
    #summands=np.power(ratings_actual-ratings_predict,2)
    mse=np.sum(np.power(ratings_actual-ratings_predict,2))/len(ratings_predict)
    return np.sqrt(mse)
   
def r_squared(ratings_predict,ratings_actual):
    #ratings_predict=svd_predict(X,omega,omega0,V)
    nom=np.sum(np.power(ratings_actual-ratings_predict,2))
    denom=np.sum(np.power(ratings_actual-np.mean(ratings_actual,axis=0),2))
    return 1-nom/denom

In [23]:
# function to create a list containing mini-batches 
def create_mini_batch(data,i, batch_size): 
    mini_batch = data.tocsr()[i * batch_size:(i + 1)*batch_size, :] 
    X_mini = mini_batch[:, :-1] 
    Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        
    if data.shape[0] % batch_size != 0: 
        mini_batch = data[i * batch_size:data.shape[0]] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
       
    return X_mini,Y_mini

def shuffle_data(matrix):
    indices = np.arange(matrix.shape[0]) #gets the number of rows 
    np.random.shuffle(indices)
    shuffled_matrix = matrix[list(indices)] 

def mini_batch_gradient_descent(X_train,Y_train,K,learning_rate,num_epoches,batch_size):
    omega=np.random.sample((X_train.shape[1],1)) 
    #omega=np.array([0.5]*(X_train.shape[1]))
    omega0=0.5
    #V=np.random.sample((X_train.shape[1],K))
    V=(np.array([0.5]*(X_train.shape[1]*K)).reshape(X_train.shape[1],K))
    r2_tr=[]
    rmse_tr=[]
   
    it=0
    for it in range(num_epoches): 
        #data = sparse.hstack((X_train, Y_train)).tocsr()
        #np.random.shuffle(data) #shuffle at each epoch (not every batch creation)
        #shuffle_data(data)
        X_train,Y_train=shuffle(X_train,Y_train)
        n_minibatches = X_train.shape[0] // batch_size 
        i = 0
        for i in range(n_minibatches + 1):
            X_mini=X_train.tocsr()[i * batch_size:(i + 1)*batch_size, :] 
            y_mini=Y_train[i * batch_size:(i + 1)*batch_size,:]
            #print(np.shape(X_mini))   
            #print(np.shape(y_mini)) 
            N=X_mini.shape[0] # 24053764
            #PARAMETERS
            sum_const=X_mini.dot(V)
            ratings_predict=svd_predict(X_mini,omega,omega0,V,sum_const)
            error=y_mini-ratings_predict #(24053764,1)
            omega=omega+learning_rate*2*((X_mini.T).dot(error))/N #(475257,1)
            omega0=omega0+learning_rate*2*np.sum(error)/N #(1,1)
            V = V + learning_rate*2*((X_mini.T.dot(np.multiply(error,sum_const)))-np.multiply(V, X_mini.T.dot(error)))/N
#             for l in range(K):
#                 for j in range (N):
#                     Xj_col=X_mini.tocsr()[:,j]#(24053764, 1)
#                     dV1=np.multiply(Xj_col,np.sum(X_mini*V[j][l],axis=1).reshape(-1,1))
                
#                     dV2=V[j][l]*Xj_col.power(2)#(24053764, 1)
#                     dV=(dV1-dV2)
                    
#                     dV=(dV1-dV2)
#                     a=np.asarray((error.T).dot(dV)).reshape(-1,1)
#                     print(a)
#                     print(V[j][l]+a)
#                     #print(np.shape(dV))
#                     #print((learning_rate*2*((error.T).dot(dV))/N).to_dense)
#                     #print (V)
#                     #print(np.shape(V[j][l].reshape(-1))
#                     #print(np.shape(V[j][l].reshape(1,-1).sum(learning_rate*2*(error.T).dot(dV).reshape(1,-1)/N)))
                    
#                     V[j][l]=V[j][l]+learning_rate*2*((error.T).dot(dV))/N #(1,1)
            ratings_predict=svd_predict(X_mini,omega,omega0,V,sum_const)
            r2_tr.append(r_squared(ratings_predict, y_mini))
            rmse_tr.append(root_mean_square_error(ratings_predict, y_mini))
            #print(rmse_tr)
            #print(r2_tr)
    return omega,omega0,V

# error=ratings-svd_predict(X,omega,omega0,V)
# N=X.shape[0]
# learning_rate=0.001
# omega=omega+learning_rate*2*((X.T).dot(error))/N #(475257,1)
# omega0=omega0+learning_rate*2*np.sum(error)/N
# for l in range(3):
#     for j in range (N):
#         Xj_col=X.tocsr()[:,j] #(24053764, 1)
#         #print(np.shape(np.multiply(Xj_col,np.sum(X*V[j][l],axis=1).reshape(-1,1)))) #(24053764, 1)
#         #print(np.shape(V[j][l]*np.square(Xj_col)))
#         dV=np.multiply(Xj_col,np.sum(X*V[j][l],axis=1).reshape(-1,1))-V[j][l]*np.square(Xj_col)
#         #print(np.shape((error.T).dot(dV))) #(1, 1)
#         V[j][l]=V[j][l]+learning_rate*2*((error.T).dot(dV))/N #(1,1)

In [24]:
r2=[]
rmse=[]
omega=[]
omega0=[]
r2_train=[]
rmse_train=[]
cross_val_parts_amount=5
cross_val_part_size=X.shape[0]//cross_val_parts_amount
item=0
for item in range(cross_val_parts_amount):
    X_test=X.tocsr()[item*cross_val_part_size:(item+1)*cross_val_part_size,:]
    Y_test=ratings[item*cross_val_part_size:(item+1)*cross_val_part_size]
    X_train=vstack([X.tocsr()[0:(item)*cross_val_part_size,:], X.tocsr()[(item+1)*cross_val_part_size:X.shape[0],:]], 'csr') 
    #Y_train=np.delete(ratings,ratings[item*cross_val_part_size:(item+1)*cross_val_part_size])
    #Y_train=ratings.drop(ratings.index[item*cross_val_part_size:(item+1)*cross_val_part_size])
    Y_train=np.vstack((ratings[0:item*cross_val_part_size],ratings[(item+1)*cross_val_part_size:ratings.shape[0]]))
    #print(np.shape(X_train))   
    #print(np.shape(Y_train))   
    start = time.time()
    omega_item,omega0_item,V_item=mini_batch_gradient_descent(X_train,Y_train,K=3,learning_rate = 0.01,num_epoches=3,
                                                            batch_size=100000)
    stop = time.time()
    duration = stop-start
    print(duration)
    sum_const=X_test.dot(V_item)                     
    ratings_predict=svd_predict(X_test,omega_item,omega0_item,V_item,sum_const)
    r2.insert(item,r_squared(ratings_predict,Y_test))
    rmse.insert(item,root_mean_square_error(ratings_predict,Y_test))
    sum_const=X_train.dot(V_item)                     
    ratings_predict=svd_predict(X_train,omega_item,omega0_item,V_item,sum_const)                    
    r2_train.insert(item,r_squared(ratings_predict,Y_train))
    rmse_train.insert(item,root_mean_square_error(ratings_predict,Y_train))
    print(r2[item],rmse[item],r2_train[item],rmse_train[item])

471.74917697906494
-0.15281695097850712 1.1646612214560201 -0.11561538536681937 1.1463648757897293
481.4578561782837
-0.1531907358813609 1.1661001236333077 -0.1192288546677085 1.1478479932780807
478.0649871826172
-0.14495764794730603 1.1677591551373108 -0.12369158337504182 1.1487229854198209
541.4026155471802
-0.1326883210204901 1.1528585049059301 -0.11484886233758407 1.1462018500405524
519.696811914444
-0.1300558592810519 1.1481567942006532 -0.1265319713873574 1.153189033653985


In [25]:
# initialise data of lists. 
data = {'1':[rmse[0],r2[0],rmse_train[0],r2_train[0]], 
        '2':[rmse[1],r2[1],rmse_train[0],r2_train[1]],
        '3':[rmse[2],r2[2],rmse_train[2],r2_train[2]],
        '4':[rmse[3],r2[3],rmse_train[3],r2_train[3]],
        '5':[rmse[4],r2[4],rmse_train[4],r2_train[4]],
        'E':[np.mean(rmse),np.mean(r2),np.mean(rmse_train),np.mean(r2_train)],
        'SD':[np.std(rmse),np.std(r2),np.std(rmse_train),np.std(r2_train)]} 
 
# Creates pandas DataFrame. 
df_res = pd.DataFrame(data, index =['RMSE', 'R^2','RMSE train','R^2 train']) 

df_res

,1,2,3,4,5,E,SD
RMSE,1.164661,1.166100,1.167759,1.152859,1.148157,1.159907,0.007879
R^2,-0.152817,-0.153191,-0.144958,-0.132688,-0.130056,-0.142742,0.009773
RMSE train,1.146365,1.146365,1.148723,1.146202,1.153189,1.148465,0.002542
R^2 train,-0.115615,-0.119229,-0.123692,-0.114849,-0.126532,-0.119983,0.004531


In [26]:
cross_val_part_size=X.shape[0]//(cross_val_parts_amount)
cross_val_part_size
j=0
for j in range(cross_val_parts_amount):
    X_test=X.tocsr()[item*cross_val_part_size:(item+1)*cross_val_part_size,:]
    print(np.shape(X_test))

(20096101, 497959)
(20096101, 497959)
(20096101, 497959)
(20096101, 497959)
(20096101, 497959)
